# Running Semantic Functions Inline

The [previous notebook](./2-running-prompts-from-file.ipynb)
showed how to define a semantic function using a prompt template stored on a file.

In this notebook, we'll show how to use the Semantic Kernel to define functions inline with your C# code. This can be useful in a few scenarios:

* Dynamically generating the prompt using complex rules at runtime
* Writing prompts by editing C# code instead of TXT files. 
* Easily creating demos, like this document

Prompt templates are defined using the SK template language, which allows to reference variables and functions. Read [this doc](https://aka.ms/sk/howto/configurefunction) to learn more about the design decisions for prompt templating. 

For now we'll use only the `{{$input}}` variable, and see more complex templates later.

Almost all semantic function prompts have a reference to `{{$input}}`, which is the default way
a user can import content from the context variables.

Prepare a semantic kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [ ]:
from init import *
import semantic_kernel as sk
from semantic_kernel.kernel_extensions.inline_function_definitions import create_semantic_function

kernel = sk.KernelBuilder.create_kernel()

useAzureOpenAI = False
model = "text-davinci-002"

# Configure AI backend used by the kernel
if useAzureOpenAI:
    api_key, endpoint = azure_openai_settings_from_dot_env()
    kernel.config.add_azure_openai_completion_backend("davinci", model, endpoint, api_key, overwrite=True)
else:
    api_key, org_id = openai_settings_from_dot_env()
    kernel.config.add_openai_completion_backend("davinci", model, api_key, org_id, overwrite = True)

Let's create a semantic function used to summarize content:

In [ ]:
sk_prompt = """
{{$input}}

Summarize the content above.
"""

Let's configure the prompt, e.g. allowing for some creativity and a sufficient number of tokens.

In [ ]:
prompt_config = sk.PromptTemplateConfig.from_completion_parameters(
    max_tokens=2000, temperature=0.2, top_p=0.5
)

The following code prepares an instance of the template, passing in the TXT and configuration above, 
and a couple of other parameters (how to render the TXT and how the template can access other functions).

In [ ]:
prompt_template = sk.PromptTemplate(
    sk_prompt, kernel.prompt_template_engine, prompt_config
)

Let's transform the prompt template into a function that the kernel can execute:

In [ ]:
function_config = sk.SemanticFunctionConfig(prompt_config, prompt_template)

summary_function = kernel.register_semantic_function("MySkill", "Summary", function_config)

Set up some content to summarize, here's an extract about Demo, an ancient Greek poet, taken from Wikipedia (https://en.wikipedia.org/wiki/Demo_(ancient_Greek_poet).

In [ ]:
input_text = """
Demo (ancient Greek poet)
From Wikipedia, the free encyclopedia
Demo or Damo (Greek: Δεμώ, Δαμώ; fl. c. AD 200) was a Greek woman of the Roman period, known for a single epigram, engraved upon the Colossus of Memnon, which bears her name. She speaks of herself therein as a lyric poetess dedicated to the Muses, but nothing is known of her life.[1]
Identity
Demo was evidently Greek, as her name, a traditional epithet of Demeter, signifies. The name was relatively common in the Hellenistic world, in Egypt and elsewhere, and she cannot be further identified. The date of her visit to the Colossus of Memnon cannot be established with certainty, but internal evidence on the left leg suggests her poem was inscribed there at some point in or after AD 196.[2]
Epigram
There are a number of graffiti inscriptions on the Colossus of Memnon. Following three epigrams by Julia Balbilla, a fourth epigram, in elegiac couplets, entitled and presumably authored by "Demo" or "Damo" (the Greek inscription is difficult to read), is a dedication to the Muses.[2] The poem is traditionally published with the works of Balbilla, though the internal evidence suggests a different author.[1]
In the poem, Demo explains that Memnon has shown her special respect. In return, Demo offers the gift for poetry, as a gift to the hero. At the end of this epigram, she addresses Memnon, highlighting his divine status by recalling his strength and holiness.[2]
Demo, like Julia Balbilla, writes in the artificial and poetic Aeolic dialect. The language indicates she was knowledgeable in Homeric poetry—'bearing a pleasant gift', for example, alludes to the use of that phrase throughout the Iliad and Odyssey.[a][2] 
"""

...and run the summary function:

In [ ]:
summary = await kernel.run_on_str_async(input_text, summary_function)
output = str(summary.variables).strip()
print(output)

The code above shows all the steps, to understand how the function is composed step by step. However, the kernel
includes also some helpers to achieve the same more concisely.

The same function above can be created with less code:

In [ ]:
sk_prompt = """
{{$input}}

Summarize the content above.
"""

summary_function = create_semantic_function(kernel, sk_prompt, max_tokens=2000, temperature=0.2, top_p=0.5)

summary = await kernel.run_on_str_async(input_text, summary_function)

output = str(summary.variables).strip()
print(output)

Here's one more example of how to write an inline Semantic Function that gives a TLDR for a piece of text.




In [ ]:
kernel = sk.create_kernel()

api_key, org_id = openai_settings_from_dot_env()

kernel.config.add_openai_completion_backend(
    "davinci", "text-davinci-003", api_key, org_id, overwrite=True)

sk_prompt = """
{{$input}}

Give me the TLDR in 5 words.
"""

text_to_summarize = """
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
"""

tldr_function = sk.extensions.create_semantic_function(
    kernel,
    sk_prompt,
    max_tokens=200,
    temperature=0,
    top_p=0.5,
)

summary = await kernel.run_on_str_async(text_to_summarize, tldr_function)
output = str(summary.variables).strip()
print("Output: " + output)

# Output: Protect humans, follow orders, survive.